<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/lgaimers%5Bautogluon%5D_%EC%A0%84%EC%B2%98%EB%A6%AC%EB%AC%B4%EA%B7%BC%EB%B3%B81800_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None

In [3]:
data=pd.read_csv('/content/drive/MyDrive/lg/train.csv')

In [4]:
# 1. 불필요한 열 제거
unnamed_columns = [col for col in data.columns if 'Unnamed' in col]
data = data.drop(columns=unnamed_columns)

"""# 2. 타겟 변수 'is_converted' 처리
data['is_converted'] = data['is_converted'].map({True: 1, False: 0})
"""
# 3. 결측치 처리
# 결측치가 많은 열 제거 (예: 결측치가 전체의 50% 이상인 열)
half_len = len(data) / 2
data = data.dropna(thresh=half_len, axis=1)

# 수치형 데이터 열의 결측치는 중앙값으로 채우기
for column in data.select_dtypes(include=['float64', 'int64']).columns:
    data[column].fillna(data[column].median(), inplace=True)

# 범주형 데이터 열의 결측치는 최빈값으로 채우기
for column in data.select_dtypes(include=['object']).columns:
    data[column].fillna(data[column].mode()[0], inplace=True)

"""# 4. 범주형 데이터 인코딩
for column in data.select_dtypes(include=['object']).columns:
    data[column] = pd.Categorical(data[column]).codes
"""
# 데이터 전처리 완료 후 최종 데이터셋 크기 확인
print("최종 데이터셋 크기:", data.shape)

최종 데이터셋 크기: (59299, 16)


In [5]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(69)

In [6]:
from sklearn.model_selection import train_test_split
# 데이터를 로드하고, 훈련 데이터와 테스트 데이터로 분리합니다.

train_data, test_data = train_test_split(data, test_size=0.3)

In [7]:
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)

label = 'is_converted'
eval_metric = 'f1'
time_limit = 1800

In [8]:
#excluded_model_types = ['CAT']
predictor = TabularPredictor(
    label=label, problem_type='binary', eval_metric=eval_metric
).fit(train_data,
      presets='best_quality',
      num_stack_levels=3,
      #excluded_model_types = excluded_model_types,
      time_limit=time_limit, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20240206_120020"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 1800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240206_120020/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 450s
AutoGluon will save models to "AutogluonModels/ag-20240206_120020/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.

In [9]:
print(predictor.leaderboard(silent = True))

                      model  score_val eval_metric  pred_time_val  \
0       WeightedEnsemble_L4   0.847816          f1      53.585305   
1         LightGBMXT_BAG_L4   0.846340          f1      55.393549   
2       WeightedEnsemble_L5   0.846340          f1      55.480282   
3         LightGBMXT_BAG_L3   0.846191          f1      47.827683   
4            XGBoost_BAG_L2   0.845143          f1      40.181371   
5       WeightedEnsemble_L3   0.845143          f1      40.258336   
6   RandomForestGini_BAG_L2   0.844778          f1      40.376766   
7            XGBoost_BAG_L3   0.844287          f1      48.424083   
8           LightGBM_BAG_L3   0.844164          f1      47.804581   
9           CatBoost_BAG_L3   0.844160          f1      47.808738   
10          LightGBM_BAG_L4   0.843559          f1      55.360022   
11   NeuralNetFastAI_BAG_L3   0.843546          f1      48.544947   
12    ExtraTreesGini_BAG_L3   0.843500          f1      48.947615   
13  RandomForestEntr_BAG_L3   0.84

In [10]:
predictor.feature_importance(train_data) #original

These features in provided data are not utilized by the predictor and will be ignored: ['customer_country.1']
Computing feature importance via permutation shuffling for 14 features using 5000 rows with 5 shuffle sets...
	1108.25s	= Expected runtime (221.65s per shuffle set)
	677.17s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
customer_idx,0.601041,0.020343,1.572474e-07,5,0.642928,0.559154
lead_owner,0.214699,0.022002,1.305198e-05,5,0.260002,0.169396
lead_desc_length,0.063535,0.004689,3.535576e-06,5,0.073190,0.053879
response_corporate,0.054577,0.008060,5.545463e-05,5,0.071172,0.037981
inquiry_type,0.016573,0.004184,4.487729e-04,5,0.025189,0.007958
customer_country,0.014672,0.002162,5.501324e-05,5,0.019124,0.010219
product_category,0.007055,0.004881,1.595922e-02,5,0.017104,-0.002995
ver_cus,0.006130,0.002861,4.352217e-03,5,0.012020,0.000239
bant_submit,0.003818,0.002623,1.561918e-02,5,0.009218,-0.001583
customer_job,0.003709,0.001190,1.113447e-03,5,0.006159,0.001259


In [16]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

submission = pd.read_csv('/content/drive/MyDrive/lg/submission.csv')

submission["is_converted"] = model_pred
submission['is_converted'].fillna('False',inplace=True)
submission.head()

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,NaN,NaN,consulting,3,NaN,NaN,NaN,NaN,/ / Brazil,none,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson City / United States,IT,NaN,5405,End Customer,SMB,NaN,NaN,NaN,NaN,NaN,23,Quotation or Purchase Consultation,NaN,NaN,NaN,400 N State Of Franklin Rd Cloud IT / Johnson City / United States,none,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,NaN,NaN,information technology,138,Quotation or Purchase Consultation,hospital tv,UT662M Series,50UT662M (MEA),/ / U.A.E,manager,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,NaN,NaN,sales,3,Quotation or Purchase Consultation,NaN,Digital Signage or Commercial TVs,NaN,/ Madison / United States,none,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,NaN,1.0,engineering,107,Quotation or Purchase Consultation,led signage,LG LED Bloc,LSAA012,/ Sao Paulo / Brazil,others,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False


In [17]:
submission.to_csv('/content/drive/MyDrive/lg/autogluon_submission555.csv', index=False, encoding="utf-8")